In [6]:
a = pd.read_csv("./train.csv")

questionn = a['question']
context = a["context"]

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

In [22]:
import pandas as pd
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
    RobertaTokenizer,
    RobertaForMaskedLM,
    RobertaForMaskedLM,
    DataCollatorForLanguageModeling,
)
import os, json, torch
import pandas as pd
from transformers import Trainer, TrainingArguments
from tqdm import tqdm
import time 
import random





def random_masking_df(input_data, tokenizer, threshold=0.9, random_state=42, shuffle=True):
    origin_data = input_data.copy()
    if shuffle==True:
        origin_data = origin_data.sample(len(origin_data), random_state=random_state)
    mask_prop_in_1sentence = 0.05

    masked_new_premise_lst = []
    masked_new_hypo_lst = []
    for origin_premise, origin_hypo in tqdm(zip(origin_data['context'],origin_data['question']), total=origin_data.shape[0]):

        will_masking_prop = random.random()
        if will_masking_prop <= threshold:
            encoded_premise_ids, encoded_hypo_ids = tokenizer(origin_premise)['input_ids'], tokenizer(origin_hypo)['input_ids']
            premise_seq_len, hypo_seq_len = len(encoded_premise_ids), len(encoded_hypo_ids)
            n_masks_premise, n_masks_hypo = min(15, max(1, int(premise_seq_len*mask_prop_in_1sentence))), min(3, max(1, int(hypo_seq_len*mask_prop_in_1sentence))) # 최소 1개 ~ 최대 3개

            mask_indices = set()
            while len(mask_indices)<n_masks_premise:
                mask_indices.add(random.randrange(1, premise_seq_len-1))
            for ind in list(mask_indices):
                encoded_premise_ids[ind] = 4 # replace to mask 
            new_premise_sentence = tokenizer.decode(encoded_premise_ids).replace('[CLS]','').replace('[SEP]','').strip()
            masked_new_premise_lst.append(new_premise_sentence)

            mask_indices = set()
            while len(mask_indices)<n_masks_hypo:
                mask_indices.add(random.randrange(1, hypo_seq_len-1))
            for ind in list(mask_indices):
                encoded_hypo_ids[ind] = 4 # replace to mask 
            new_hypo_sentence = tokenizer.decode(encoded_hypo_ids).replace('[CLS]','').replace('[SEP]','').strip()
            masked_new_hypo_lst.append(new_hypo_sentence)

        else:
            masked_new_premise_lst.append(origin_premise)
            masked_new_hypo_lst.append(origin_hypo)
               
               
    origin_data = origin_data.reset_index(drop=True)
    return masked_new_hypo_lst, masked_new_premise_lst



In [23]:
que, con = random_masking_df(input_data=a, tokenizer=tokenizer)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 4192/4192 [00:10<00:00, 418.86it/s]


In [24]:
con

["1952년 [MASK]을 졸업하자 김임식 의 소개로 개성중학교에서 교편을 잡게 되었고, 이어서 1954년 부산상업고등 [MASK] 교사 로 옮겼다. 이 때 서길덕 등과 교류했다. 공대를 졸업했음에도 수학교사로 있으면서 박기택 의 권유로 부산대학교에 [MASK]강하였고, 윤 [MASK] 총장 시절인 1958년 부산대학교 공과대학 교수 로 부임하였다. 이 때 민수홍, 김만식 최내형, 정희채, 이형기 등과 교류 [MASK]였다. 이 무렵 집안의 큰아들로서 어린 동생들을 공부시켜야 하 [MASK] 등 집안을 책임지고 있 [MASK] 터에 어머니의 병 구 [MASK]으로 살림이 어려워졌고, 1962년에는 결국 어머니 [MASK] 여의었다. [UNK] n [UNK] n19 [MASK]년 『 저탄소강의 피로에 의한 슬립밴드의 거동과 스트레인 시효에 관한 연구 』 라는 논문으로 부산대학교에서 공학박사 학위 를 받았고, 같은 해 5월에는 대한기계학회에서'피로의 통계학적 분석'이라는 제하로 춘계학술강연 [MASK] n을 하였다. 1974년 부산대학교 공과대학장을 맡았을 때는 윤천주 총장을 도와 공대 발전을 도모하였는데 [MASK] 재임 당시 국내 공업육성을 위해 정부가 공과대학 특성화를 추진할 때, [MASK]학교가 기계특성화 공대로 지정 되는 등 학교 발전에 역량을 [MASK] [MASK]다. 1979년 일본국 동북대학 공학부에서 연구하였고, 가야바교수와 친분을 쌓았다. 1983년 5월에는 한국기계공업진흥회에서 열린 대한기계학회의 고체역학 분과에서 학술강연을 하였다.",
 '19세기 문학은 대혁명으로 새로운 [MASK]의 담당자가 된 부르주아지가 과거의 제약에서 자아를 해방하여 문학적으로 표현하는 일에서 시작된다. 근대문학은 여기에서 시작한 것이라고도 할 수 있다. 19 [MASK]는 이 부르주아지와 더불어 많은 정치적 변천을 겪으면서 나아간다. 이러한 사회변동과 밀접한 영향관계를 유지하면서 문학에 있어서도 갖가지 유파가 생겨나고 또 교체해 간다. [UNK] n [UNK]